In [2]:
import torch
import torch.nn as nn
import numpy as np
import os
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

/Users/doriclink/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Using weights from https://github.com/mseitzer/srgan/



model = torch.load("./weights/srgan.pth")
generator = model["runner"]["generator"]
for k, v in generator.items():
    print(k, v.shape)


initial_conv.1.weight torch.Size([64, 3, 9, 9])
initial_conv.1.bias torch.Size([64])
initial_conv.2.weight torch.Size([64])
body.0.block.1.weight torch.Size([64, 64, 3, 3])
body.0.block.2.weight torch.Size([64])
body.0.block.2.bias torch.Size([64])
body.0.block.2.running_mean torch.Size([64])
body.0.block.2.running_var torch.Size([64])
body.0.block.3.weight torch.Size([64])
body.0.block.5.weight torch.Size([64, 64, 3, 3])
body.0.block.6.weight torch.Size([64])
body.0.block.6.bias torch.Size([64])
body.0.block.6.running_mean torch.Size([64])
body.0.block.6.running_var torch.Size([64])
body.1.block.1.weight torch.Size([64, 64, 3, 3])
body.1.block.2.weight torch.Size([64])
body.1.block.2.bias torch.Size([64])
body.1.block.2.running_mean torch.Size([64])
body.1.block.2.running_var torch.Size([64])
body.1.block.3.weight torch.Size([64])
body.1.block.5.weight torch.Size([64, 64, 3, 3])
body.1.block.6.weight torch.Size([64])
body.1.block.6.bias torch.Size([64])
body.1.block.6.running_mean tor